In [13]:
import os
# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas as pd
import numpy as np
import sys


from evaluation.metric import computeAccuracy, computeF1Score, computeAUC, computeAUCBehavior
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline

0.17


In [14]:
# clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa_initial = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
clf_behavior_initial = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pd.read_csv("./data/alcohol_training_instances.csv")

Prepare the behavior level data

In [15]:
# appending labels based on whether tweet label contains phrase
behavior_labels = []
behavior_index_in_df = []

fpl = "'first_person_level'"

# extracting FP levels and constructing subset vector simultaneously
for i in range(len(df)):
    if fpl in df.labels[i]:
        behavior_labels.append(int(df.labels[i][df.labels[i].find(fpl) + len(fpl) + 2]))
        behavior_index_in_df.append(1)
    else:
        behavior_index_in_df.append(0)

behavior_labels = list(map(int, behavior_labels))
print(behavior_labels[:20])
# print(subset_vec_alc_fp[:10])
print('length of first person level labels: ', len(behavior_labels)) # 6357
print(behavior_index_in_df[:20])
print('length of first person level index in df: ', len(behavior_index_in_df)) # 15651
index_behavior = list(np.where(behavior_index_in_df)[0])
print(index_behavior[:15])
print('length of first person level df index: ', len(index_behavior)) # 6357
df_behavior = df.iloc[index_behavior,]
# df_fp.head
# fp_labels = np.asarray(fp_labels)

# change label 3 to label 0
behavior_labels = [0 if x == 3 else x for x in behavior_labels] 

# train test dataset split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_behavior, behavior_labels, test_size=0.2, random_state=26)

[0, 0, 2, 2, 1, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0]
length of first person level labels:  6357
[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]
length of first person level index in df:  15651
[2, 5, 6, 7, 11, 13, 17, 18, 21, 22, 23, 24, 26, 27, 28]
length of first person level df index:  6357


In [16]:
print(len(behavior_labels))
print(len(df_behavior))
# behavior_labels[:30]
# for i in behavior_labels:
#     if i ==3:
#         print(True)

6357
6357


Filter out those with reflective label

In [17]:
reflective_index_in_behavior = []
for i in range(len(behavior_labels)):
    if behavior_labels[i] == 2:
        reflective_index_in_behavior.append(i)
df_reflective = df_behavior.iloc[reflective_index_in_behavior,]


In [18]:
df_reflective

,_id,created_at,labels,predict,text,user,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.statuses_count,user.verified
6,556e16c2d6dfbb46288139a7,Tue Jun 02 20:49:44 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.761665,Its only Tuesday and I need a drink or four.,{'created_at': 'Sun Jan 24 00:57:08 +0000 2010...,Sun Jan 24 00:57:08 +0000 2010,1522,211.0,516.0,5300.0,False
7,556e173dd6dfbb4628813e3b,Tue Jun 02 20:51:47 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.622639,jess told me no puking this friday.. so that m...,{'created_at': 'Fri Dec 06 05:58:03 +0000 2013...,Fri Dec 06 05:58:03 +0000 2013,6814,4573.0,408.0,7859.0,False
24,556e3732d6dfbb6d1e39c3aa,Tue Jun 02 23:08:07 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.639314,I'll be fine just drinking my wine,{'created_at': 'Sat May 26 15:30:17 +0000 2012...,Sat May 26 15:30:17 +0000 2012,7529,1560.0,695.0,91150.0,False
32,556e3d1ed6dfbb6d1e39fe92,Tue Jun 02 23:33:23 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.540555,Wtf was I drinking Sunday?😂🔫🙊 http://t.co/3c2m...,{'created_at': 'Tue Nov 02 17:30:02 +0000 2010...,Tue Nov 02 17:30:02 +0000 2010,3466,309.0,173.0,11567.0,False
44,556e4632d6dfbb7a8de424ae,Wed Jun 03 00:12:08 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.735108,I can go through the week without drinking but...,{'created_at': 'Sat Oct 03 23:15:08 +0000 2009...,Sat Oct 03 23:15:08 +0000 2009,679,1469.0,1127.0,31666.0,False
48,556e4757d6dfbb7a8de431b6,Wed Jun 03 00:17:00 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.561709,Drinking a Café Deth by @revbrewchicago at @ro...,{'created_at': 'Tue Nov 11 21:58:49 +0000 2008...,Tue Nov 11 21:58:49 +0000 2008,9,80.0,314.0,4574.0,False
66,556e52fad6dfbb077d48274b,Wed Jun 03 01:06:40 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.615353,#NoteToSelf Don't drink on a Monday..😣,{'created_at': 'Fri Mar 28 18:52:20 +0000 2014...,Fri Mar 28 18:52:20 +0000 2014,7618,71.0,77.0,3869.0,False
110,556e7502d6dfbb1bcf16e518,Wed Jun 03 03:31:51 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.750131,Haven't been drunk or high in a while.,{'created_at': 'Sun May 02 07:36:43 +0000 2010...,Sun May 02 07:36:43 +0000 2010,9738,1288.0,1008.0,66286.0,False
115,556e772fd6dfbb1bcf17048e,Wed Jun 03 03:41:08 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.730232,Niggas recoding process is drinking and smokin...,{'created_at': 'Sun Jun 19 13:01:14 +0000 2011...,Sun Jun 19 13:01:14 +0000 2011,31,2178.0,34.0,73612.0,False
117,556e796ed6dfbb1bcf1723f3,Wed Jun 03 03:50:44 +0000 2015,"{'alcohol': 1, 'first_person_level': 2, 'first...",0.524286,Peach vodka is horrendous tasting,{'created_at': 'Fri Oct 16 14:44:13 +0000 2009...,Fri Oct 16 14:44:13 +0000 2009,6019,545.0,483.0,38651.0,False


In [19]:
print('We got %d reflective data' %(len(df_reflective)))

We got 1394 reflective data


In [27]:
print(pd.max_rows())

AttributeError: module 'pandas' has no attribute 'max_rows'

In [41]:
pd.set_option('max_colwidth',120)
# pd.set_option('height',1000)
# pd.set_option('display.max_rows',500)
# pd.set_option('display.max_columns',500)
# pd.set_option('display.width',1000)

# pd.set_option('display.max_rows', 500)  #最大行数
# pd.set_option('display.max_columns', 500)    #最大列数
# pd.set_option('display.width', 4000)        #页面宽度


df_reflective.text.head

<bound method NDFrame.head of 6                                                                                   Its only Tuesday and I need a drink or four.
7                                                  jess told me no puking this friday.. so that means no getting drunk... ouch 😩
24                                                                                            I'll be fine just drinking my wine
32                                                                          Wtf was I drinking Sunday?😂🔫🙊 http://t.co/3c2mz1Uz8L
44                 I can go through the week without drinking but soon as the weekend hit mannnnn I feel that shit is owed to me
48                                               Drinking a Café Deth by @revbrewchicago at @rootspizza — http://t.co/ni9qZl4N9S
66                                                                                        #NoteToSelf Don't drink on a Monday..😣
110                                                                